In [ ]:
%run -i ../python/common.py
UC_SKIPTERMS=True
%run -i ../python/ln_preamble.py

# UC-SLS Lecuture 17 : In to the Light - C Intro

In [ ]:
# setup for sumit examples
appdir=os.getenv('HOME')
appdir=appdir + "/cintro"
#print(movdir)
output=runTermCmd("[[ -d " + appdir + " ]] &&  rm -rf "+ appdir + 
             ";mkdir " + appdir + 
             ";cp ../src/Makefile ../src/setup.gdb ../src/myfunc0.c ../src/usecsumit1.S ../src/csumit1.c ../src/10.nums ../src/misc.h ../src/loop.c  ../src/hello.s " + appdir)

display(Markdown('''
- create a directory `mkdir cintro; cd cintro`
- copy examples
- add a `Makefile` to automate assembling and linking
    - we are going run the commands by hand this time to highlight the details
- add our `setup.gdb` to make working in gdb easier
- normally you would want to track everything in git
'''))
TermShellCmd("ls " + appdir)

## What are some the downsides of Assembly Programming?


1. The human burden of assembly programming and programmer lock in
2. The code we write is locked to a specific CPU
3. The code we write is locked to a specific OS


## High level languages

- downsides?
- lets look at python "Hello World"
     - How many instructions for our "hello.s" : < 10 instructions
     - `python -c print("Hello World")`: 10's, 100's, 1000's, 10,000, 100,000's, 1,000,000's, > 10,000,000's ???
     - gdb 
         
         - `display /1i $pc`
         - `starti print("Hello World")`
         - `while 1`
         - `   stepi`
         - `end`
     - https://github.com/python/cpython

## The ToolChain

<center>
<img src="../images/C-001/C-001.001.png" >
</center>

<center>
<img src="../images/C-001/C-001.002.png" >
</center>

<center>
<img src="../images/C-001/C-001.003.png" >
</center>

<center>
<img src="../images/C-001/C-001.004.png" >
</center>

## ToolChain in action: A simple C version of sumit

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/csumit1.c", 
    lang="c", 
    title="<b>CODE: csumit1.c",
    h="100%", 
    w="107em"
)))

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/usecsumit1.S", 
    lang="c", 
    title="<b>CODE: usecsumit1.S",
    h="100%", 
    w="107em"
)))

### Run Assembler on usecsumit1.S
**Use assembler as expected to `usecsumit1.S` $\rightarrow$ `usecsumit1.o`

In [ ]:
TermShellCmd("[[ -a usecsumit1.o ]] && rm usecsumit1.o;make ASFLAGS='' usecsumit1.o", cwd=appdir, prompt='')

In [ ]:
TermShellCmd("ls -l usecsumit1.o", cwd=appdir, noposttext=True)

### Run Compiler on csumit.c

**A New Step - compile for `csumit1.c` $\rightarrow$ `csumit1.s`**

In [ ]:
TermShellCmd("[[ -a csumit1.s ]] && rm csumit1.s; make csumit1.s", cwd=appdir, prompt='')

In [ ]:
TermShellCmd("ls -l csumit1.s", cwd=appdir, noposttext=True)

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/csumit1.c", 
    lang="c", 
    title="<b>CODE: csumit1.c",
    h="100%", 
    w="107em"
)))

In [ ]:
display(Markdown(FileCodeBox(
    file=appdir + "/csumit1.s", 
    lang="gas", 
    title="<b>CODE: csumit1.s",
    h="100%", 
    w="107em"
)))

### Run Assembler on csumit1.s

**Now same old same old for `csumit1.s` $\rightarrow$ `csumit1.o`**

In [ ]:
TermShellCmd("[[ -a csumit1.o ]] && rm csumit1.o;make ASFLAGS='' csumit1.o", cwd=appdir, prompt='')

In [ ]:
TermShellCmd("ls -l csumit1.o", cwd=appdir, noposttext=True)

### Linking it all together

**Use Linker to create executable `usecsumit1.o` $+$ `csumit1.o` $\rightarrow$ `usecsumit1`**

In [ ]:
TermShellCmd("ls -l usecsumit1.* csumit1.*", cwd=appdir, noposttext=True)

In [ ]:
TermShellCmd("[[ -a usecsumit1 ]] && rm usecsumit1;make LDFLAGS='' usecsumit1", cwd=appdir, prompt='')

In [ ]:
TermShellCmd("ls -l usecsumit1* csumit1.*", cwd=appdir, noposttext=True)

### Run our executable

In [ ]:
TermShellCmd("./usecsumit1", cwd=appdir, noposttext=True)

#### Exercises 

- add io to usecsumit1
  - read binary input in from stdin into XARRAY
  - add a sum memory variable 
  - after call to sumit move the result in rax into the sum memory variable 
  - write value of sum memory variable to stdout
  

## Remember `objdump` is another useful tool

- Let's us work directly with object files both relocatable and executables
- Let's us see what's inside and where and what the loader is supposed to do
- Has similar capabilities to debugger like `gdb` 
  - but often easer to use when you just want to look at things and not actually debug/run things. 

In [ ]:
TermShellCmd("objdump -M intel -S -d usecsumit1", cwd=appdir, noposttext=True)

### OF COURSE THE DEBUGGER IS STILL OUR BEST FRIEND

- we can do everything we were doing before
- examine memory
- list assembly source
- disassemble
- set break points
- But now if we allow the assembler and linker to produce debug info `-g` then
  - we can work with C source level
    - list C source that corresponds to the opcodes
    - set break points via C source lines
    - examine C variables with the debugger knowing the correct types 
      - 1, 2, 4 or 8 byte types
      - pointers vs variables
      - signed versus unsigned
      - and support for complex heterogeneous types (we will see this later)

In [ ]:
showDT()

- `gcc --static -g -nostartfiles -nostdlib  csumit1.c usecsumit1.S -o sum`
  - here we let the compiler driver do all the steps for us
    - it creates the necessary "intermediary" files (.s and .o files) and removes then when done
    - it runs the "compiler", "assembler" and "linker" for us : use `-v` flags to sees this happend
  - in our case since we are not using the C library or "runtime" we suppress their use 
    - rather we want to provide our own `_start`
    - we are just using the compiler to avoid writing all our code in assembly
- we can now `list sumit`
- set breakpoints on C lines `break 8`
- `disassemble sumit`
- print and examine C variables
  - `p i`
  - `p sum`
  - `p XARRAY`
  - `p XARRAY[0]`
- ask what the type of a variable is 
  - `whatis XARRAY`

**In other words our use of a particular memory location is now clear and explicit**

<center>
<img src="../images/C-001/C-001.014.png" >
</center>

<center>
<img src="../images/C-001/C-001.015.png" >
</center>

<center>
<img src="../images/C-001/C-001.016.png" >
</center>

<center>
<img src="../images/C-001/C-001.017.png" >
</center>

## Before we can get really get going

### Our particular compiler : GCC

https://gcc.gnu.org/onlinedocs/gcc/index.html#SEC_Contents
  - Example of one of the "C" Standards : https://www.open-std.org/jtc1/sc22/wg14/www/docs/n2310.pdf

- is the back bone of the GNU Linux software environment
- however it is by no means the only C compiler tool chain
  - llvm
  - xlc
  - Microsoft's C compiler
  - and many more

### Controlling the compiler with its command line options

- The C compiler is a very sophisticated program and has many options that control the assembly code it creates
- we are going to use options so that make it easier to read the code it creates
  - normally the code it creates does not target human readability
  - It usually includes a lot of extra directives to provide the debugger and other tools with more information
  - and by default wants to keep compilation fast

https://gcc.gnu.org/onlinedocs/gcc/Optimize-Options.html

#### Optimization level `-O`
  - by default if no optimization level is given to `gcc` the compiler suite we are using
      - produces code that is suited for use and manipulation in the debugger
        - this code sacrifices efficiency so that it is easy to work at the "C" source level
      - we will use `-Os` to set the optimization level to `s` which
        - does many optimizations but tries to keep the code "small" 

#### Turning off features that we don't need

- We also going turn off some features designed to exploit features of the Intel processor that make the code more secure
  - `-fcf-protection=none`
- We are going to turn off certain debugging directives 
  - `-fno-asynchronous-unwind-tables`
- We are going to turn off support for dynamic (load/runtime) relocation and linking 
  -  `-fno-pic`  : turn off generation of position independent code
  - `-static`    : force static linking 
  
#### We are going to force ourselves to write good code making all warning errors
  - `-Werror`
  - In general you should never have warning in your code - it is a sign that you don't know what you are doing
  
#### Explicitly produce assembly files
To have the compiler driver only preprocess and compile to produce .s files
  - `-S`

#### Generate intel syntax assembly code
To have it generate intel syntax 
  - `-masm=intel`
  
#### Eg. 

In [ ]:
TermShellCmd("[[ -a myfunc0.s ]] && rm myfunc0.s; make myfunc0.s", cwd=appdir, prompt='')

## Compiler Driver vs Compiler and "Building"

- The term compiler is used in multiple ways
  1. component of the tool chain that translates C into assembly
     - or whatever language it was built for (eg. Fortran, etc) 
  2. The master command of the the tool chain that 
     - a meta command that knowns how to invoke each of the components of the tool chain
     - passed a default set of options to the components
       - you can override these
     - default behavior is to try and run all steps of the toolchain to try and create and executable from the files specified
       - eg. `gcc my1.c my2.c myasm.S neuralnet.o -o myexe`
       - if any errors occurs it will stop and report them from which ever component failed 
       - it creates all intermediary files as temporaries and removes them once done 
         - eg. it creates `.s` and `.o` files as it needs too and deletes them when it is done
         - you can override this behavior 
           - `-E` only pre-process 
           - `-S` pre-process and compile to produce `.s`
           - `-c` pre-process, compile and assemble to produce `.o`
  3. The most common use is to separated out creating `.o` from linking into an executable eg.
      - Your program is composed from a mixture of source files:
        - my1.c, my2.c, myasm.S
        - for these you would separately "compile" each to produce a corresponding .o
          1. `gcc -c my1.c -o my1.o` (runs pre-processor, compiler, and assembler)
          2. `gcc -c my2.c -o my2.o` (runs pre-processor, compiler, and assembler)
          3. `gcc -c myasm.S -o myasm.o` (runs pre-processor and assembler)
      - Your program uses an existing object file from a library that someone gave you
        - neuralnet.o
      - Now you as a separate step you link all the object files together to produce your binary
        - `gcc my1.o my2.o myasm.o neuralnet.o -o myexe`
      - You would automate all of this with an Makefile so that when you want to "build" your executable
         - make will 
           1. Only run the necessary steps "compile" steps depending on what source files are newer than their corresponding `.o`
           2. Then it will re-link all the `.o`, updated ones and existing ones, to produce a new version of the executable

## Another Link: The C Preprocessor

<center>
<img src="../images/C-001/C-001.018.png" >
</center>

<center>
<img src="../images/C-001/C-001.019.png" >
</center>

<center>
<img src="../images/C-001/C-001.020.png" >
</center>

<center>
<img src="../images/C-001/C-001.021.png" >
</center>

<center>
<img src="../images/C-001/C-001.022.png" >
</center>

<center>
<img src="../images/C-001/C-001.023.png" >
</center>

<center>
<img src="../images/C-001/C-001.024.png" >
</center>

### Preprocessor in action

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/misc.h", 
    lang="c", 
    title="<b>CODE: misc.h",
    h="100%", 
    w="107em"
)))

In [ ]:
display(Markdown(FileCodeBox(
    file="../src/loop.c", 
    lang="c", 
    title="<b>CODE: loop.c",
    h="100%", 
    w="107em"
)))

In [ ]:
TermShellCmd(" gcc -E loop.c", cwd=appdir, noposttext=True)

- In some sense the preprocessor lets us "craft" our code into our own creature that can be customized and controlled
  -  We can now change our code by modifying and defining macros
     1. Edit "misc.h"
     2. or use ability to define and undefine macros via gcc command line options
        - https://gcc.gnu.org/onlinedocs/gcc/Preprocessor-Options.html
```
-D name
Predefine name as a macro, with definition 1.

-D name=definition
The contents of definition are tokenized and processed as if they appeared during translation phase three in a ‘#define’ directive. In particular, the definition is truncated by embedded newline characters.

If you are invoking the preprocessor from a shell or shell-like program you may need to use the shell’s quoting syntax to protect characters such as spaces that have a meaning in the shell syntax.

If you wish to define a function-like macro on the command line, write its argument list with surrounding parentheses before the equals sign (if any). Parentheses are meaningful to most shells, so you should quote the option. With sh and csh, -D'name(args…)=definition' works.

-D and -U options are processed in the order they are given on the command line. All -imacros file and -include file options are processed after all -D and -U options.

-U name
Cancel any previous definition of name, either built in or provided with a -D option.
```

In [ ]:
TermShellCmd(" gcc -D ENABLE_TRACE_LOOP -E loop.c", cwd=appdir, noposttext=True)

In [ ]:
TermShellCmd(" gcc -D ENABLE_VERBOSE -E loop.c", cwd=appdir, noposttext=True)

In [ ]:
TermShellCmd(" gcc -D ENABLE_VERBOSE -D ENABLE_TRACE_LOOP -E loop.c", cwd=appdir, noposttext=True)